In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
%config Completer.use_jedi = False

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


# Setup :
To Turn on Kaggle Auto Complet use  : ***%config Completer.use_jedi = False***

**Tasks :**
- Import data 
- Columns classification
- Split data

In [2]:
import os
X_full = pd.read_csv('../input/home-data-for-ml-course/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/home-data-for-ml-course/test.csv', index_col='Id')
# Target : 'SalePrice' vertically 
X= X_full.drop("SalePrice",axis=1)
y=X_full["SalePrice"]
cols = X.columns

In [3]:
# What types of features are in the data
# Find Important categorical data : 
cats=[cat for cat in cols if X[cat].dtype=="object" and X[cat].nunique()<10]
nums=[num for num in cols if X[num].dtype in ['int64', 'float64']]
# Find Important categorical data : 
# Concate all
cols= cats+nums

In [4]:
# Split Data set 
from sklearn.model_selection import train_test_split as split
X_train_full, X_valid_full, y_train, y_valid = split(X, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=1)
X_train = X_train_full[cols].copy()
X_valid = X_valid_full[cols].copy()
X_test = X_test_full[cols].copy()
X_full_end=X_full[cols].copy

# Prepocessing :

Transfer the data 

- Missing values : imputer
- Categorical values : One-Hot-Encoding

In [5]:
# Imputer
from sklearn.impute import SimpleImputer as SImp
# Numerical
NImp = SImp(strategy="mean")
CImp = SImp(strategy="most_frequent")
# One hot encoding 
from sklearn.preprocessing import OneHotEncoder

In [6]:
# Build the prepocessing pipeline:
from sklearn.pipeline import Pipeline

# Categorical pipeline:
CPipline=Pipeline(steps=[
    ("impute",CImp),
    ("Encoding",OneHotEncoder(handle_unknown='ignore'))
])

# Preprocessih
from sklearn.compose import ColumnTransformer
Preprocessing=ColumnTransformer(transformers=[
    ("Num",NImp,nums),
    ("Cat",CPipline,cats)
])

# Model 
XGBoost


In [7]:
from xgboost import XGBRegressor as xgb
model=xgb(n_estimators=1000,learning_rate=0.01)
# Others : Eealy stop 

# Compose the Whole Pipeline

In [8]:
pipline=Pipeline(steps=[
    ("Pre",Preprocessing),
    ("model",model)
])

# Fitting 

In [9]:
pipline.fit(X_train,y_train)

Pipeline(steps=[('Pre',
                 ColumnTransformer(transformers=[('Num', SimpleImputer(),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBath',
                                                   'HalfBath', 'BedroomAbvGr...
                              colsample_bytree

In [10]:
Vprediction=pipline.predict(X_valid)

# Evaluation

In [11]:
from sklearn.metrics import mean_absolute_error as mae
mae(Vprediction,y_valid)

15248.300901648116

# Train the whole 


In [12]:
# Train the whole 
pipline.fit(X[cols],y)

Pipeline(steps=[('Pre',
                 ColumnTransformer(transformers=[('Num', SimpleImputer(),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBath',
                                                   'HalfBath', 'BedroomAbvGr...
                              colsample_bytree

In [13]:
Aprediction=pipline.predict(X[cols])

In [14]:
mae(Aprediction,y)

4562.306862692637

# Submit

In [15]:
Tprediction=pipline.predict(X_test)

In [16]:
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': Tprediction})
output.to_csv('submission.csv', index=False)